## Script responsável por descompactar o ZIP

- Antes de executar esse script, fazer download da base do DOU no link: https://www.in.gov.br/acesso-a-informacao/dados-abertos/base-de-dados
- Há um arquivo zip para cada ano/mês/seção
- Salvar no subdiretório \downloads\Secao0<Número da secao>\ano<com 4 dígitos>. Exemplo: \downloads\Secao02\2024
- No processamento o script:
  - Descompacta todos os arquivos zips existentes no subdiretório downloads\Secao0<Número da secao>
  - Faz a leitura do XML
  - Cria a coluna Texto a partir da TextHTML, removento das tags html do corpo da portaria
  - Salva a saída no arquivo output9<seção>


In [1]:
import os
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import io
from bs4 import BeautifulSoup

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

In [3]:
def process_zip(zip_path):

    print(zip_path)
    # Lista para armazenar os dados dos artigos
    articles_data = []

    # Abre o arquivo ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Itera sobre cada arquivo no ZIP
        for file_name in zip_ref.namelist():
            if file_name.endswith('.xml'):
                try:
                    # Abre o arquivo XML
                    with zip_ref.open(file_name) as file:
                        # Lê o conteúdo do arquivo
                        content = file.read()
                        # Verifica se o conteúdo não está vazio
                        if content.strip():
                            # Parse do conteúdo XML
                            tree = ET.ElementTree(ET.fromstring(content))
                            root = tree.getroot()

                            # Itera sobre todos os elementos <article>
                            for article in root.findall('article'):
                                if article is not None:
                                    # Extrai os atributos do <article>
                                    article_data = article.attrib

                                    # Extrai os conteúdos dos elementos dentro de <body>
                                    body = article.find('body')
                                    if body is not None:
                                        for child in body:
                                            # Usa o nome do elemento como chave e seu texto como valor
                                            article_data[child.tag] = child.text.strip() if child.text else None

                                    # Adiciona o nome do arquivo aos dados do artigo
                                    article_data['file_name'] = file_name
                                    # Adiciona o nome do arquivo ZIP aos dados do artigo
                                    article_data['zip_name'] = os.path.basename(zip_path)
                                    articles_data.append(article_data)
                except ET.ParseError:
                    print(f"Erro ao analisar XML no arquivo {file_name} no ZIP {zip_path}")
                except Exception as e:
                    print(f"Erro ao processar o arquivo {file_name} no ZIP {zip_path}: {e}")

    return articles_data

In [4]:
def process_all_zips_in_directory(directory):

    print(directory)
    # Lista para armazenar os dados de todos os artigos
    all_articles_data = []

    # Itera sobre todos os arquivos no diretório e subdiretórios
    for root, dirs, files in os.walk(directory):
        print(f'Processando diretório {root}...')
        for file_name in files:
            if file_name.endswith('.zip'):
                zip_path = os.path.join(root, file_name)
                articles_data = process_zip(zip_path)
                all_articles_data.extend(articles_data)

    # Cria um DataFrame a partir dos dados de todos os artigos
    df = pd.DataFrame(all_articles_data)
    return df

In [5]:
# Função para extrair o texto de um texto HTML
def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text(separator=' ')

In [6]:
douItem = input("Informe a seção: 1, 2 ou 3")
secao = f'Secao0{douItem}'

# Caminho para o diretório contendo os arquivos ZIP
directory_path = os.path.join('./downloads/', secao)

# Processa todos os ZIPs no diretório e obtém o DataFrame
df = process_all_zips_in_directory(directory_path)

# Renomeia a coluna Texto
df.rename(columns={'Texto': 'TextoHTML'}, inplace=True)

# Cria uma nova coluna 'text_content' com o texto extraído do HTML
df['Texto'] = df['TextoHTML'].apply(extract_text_from_html)

# Exibe o DataFrame
df.shape

Informe a seção: 1, 2 ou 3 2


./downloads/Secao02
Processando diretório ./downloads/Secao02...
Processando diretório ./downloads/Secao02\2023...
./downloads/Secao02\2023\S02012023.zip
./downloads/Secao02\2023\S02022023.zip
./downloads/Secao02\2023\S02032023.zip
./downloads/Secao02\2023\S02042023.zip
./downloads/Secao02\2023\S02052023.zip
./downloads/Secao02\2023\S02062023.zip
./downloads/Secao02\2023\S02072023.zip
./downloads/Secao02\2023\S02082023.zip
./downloads/Secao02\2023\S02092023.zip
./downloads/Secao02\2023\S02102023.zip
./downloads/Secao02\2023\S02112023.zip
./downloads/Secao02\2023\S02122023.zip
Processando diretório ./downloads/Secao02\2024...
./downloads/Secao02\2024\S02012024.zip
./downloads/Secao02\2024\S02022024.zip
./downloads/Secao02\2024\S02032024.zip
./downloads/Secao02\2024\S02042024.zip
./downloads/Secao02\2024\S02052024.zip
./downloads/Secao02\2024\S02062024.zip
./downloads/Secao02\2024\S02072024.zip
./downloads/Secao02\2024\S02082024.zip
./downloads/Secao02\2024\S02092024.zip
./downloads/Seca

(337494, 28)

In [7]:
df.shape

(337494, 28)

In [8]:
df.to_parquet(f'./saida/DOU{secao}.parquet', engine='pyarrow', index=False)

In [9]:
df.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,idMateria,Identifica,Data,Ementa,Titulo,SubTitulo,TextoHTML,file_name,zip_name,Texto
0,29919584,3112_DEP_AGU_S2,9328762,DO2,Decreto de Pessoal,01/01/2023,00005:00000:00000:00000:00000:00000:00000:0000...,Atos do Poder Executivo,12,,...,20225185,DECRETO DE 31 DE DEZEMBRO DE 2022,None,None,ADVOCACIA-GERAL DA UNIÃO,None,"<p class=""titulo"">ADVOCACIA-GERAL DA UNIÃO</p>...",S02012023/529_20230101_20225185.xml.xml,S02012023.zip,ADVOCACIA-GERAL DA UNIÃO DECRETO DE 31 DE DEZE...
1,29919587,3112_DEP_MEC_S2,9328762,DO2,Decreto de Pessoal,01/01/2023,00005:00000:00000:00000:00000:00000:00000:0000...,Atos do Poder Executivo,12,,...,20225186,DECRETO DE 31 DE DEZEMBRO DE 2022,None,None,MINISTÉRIO DA EDUCAÇÃO,None,"<p class=""titulo"">MINISTÉRIO DA EDUCAÇÃO</p><p...",S02012023/529_20230101_20225186.xml.xml,S02012023.zip,MINISTÉRIO DA EDUCAÇÃO DECRETO DE 31 DE DEZEMB...
2,29919590,3112_DEP_MINFRA_S2,9328762,DO2,Decreto de Pessoal,01/01/2023,00005:00000:00000:00000:00000:00000:00000:0000...,Atos do Poder Executivo,12,,...,20225187,DECRETO DE 31 DE DEZEMBRO DE 2022,None,None,MINISTÉRIO DA INFRAESTRUTURA,None,"<p class=""titulo"">MINISTÉRIO DA INFRAESTRUTURA...",S02012023/529_20230101_20225187.xml.xml,S02012023.zip,MINISTÉRIO DA INFRAESTRUTURA DECRETO DE 31 DE ...
3,29919593,3112_DEP_MJSP_S2,9328762,DO2,Decreto de Pessoal,01/01/2023,00005:00000:00000:00000:00000:00000:00000:0000...,Atos do Poder Executivo,12,,...,20225188,DECRETO DE 31 DE DEZEMBRO DE 2022,None,None,MINISTÉRIO DA JUSTIÇA E SEGURANÇA PÚBLICA,None,"<p class=""titulo"">MINISTÉRIO DA JUSTIÇA E SEGU...",S02012023/529_20230101_20225188.xml.xml,S02012023.zip,MINISTÉRIO DA JUSTIÇA E SEGURANÇA PÚBLICA DECR...
4,29919596,3112_DEP_MMA_S2,9328762,DO2,Decreto de Pessoal,01/01/2023,00005:00000:00000:00000:00000:00000:00000:0000...,Atos do Poder Executivo,12,,...,20225189,DECRETO DE 31 DE DEZEMBRO DE 2022,None,None,MINISTÉRIO DO MEIO AMBIENTE,None,"<p class=""titulo"">MINISTÉRIO DO MEIO AMBIENTE<...",S02012023/529_20230101_20225189.xml.xml,S02012023.zip,MINISTÉRIO DO MEIO AMBIENTE DECRETO DE 31 DE D...


In [10]:
totais = df.groupby('artType').size()
print(totais)

artType
Ato                           18908
Ato Concessório                 548
Circular                          1
Decisão                         481
Decreto de Pessoal              728
Deliberação                      29
Despacho                      16842
Edital                         1739
Edital de Citação               177
Edital de Convocação            182
Edital de Intimação              40
Edital de Notificação           927
Lista de Antiguidade             39
Portaria                     287471
Portaria Conjunta              1558
Portaria Interministerial        48
Processo Disciplinar              2
Provimento                        7
Recurso Disciplinar               3
Resolução                       742
Retificação                    7010
Retificação (de Edital)          12
dtype: int64
